### metaclass:python中的黑魔法
#### 是什么?
meta包含超越、变形的含义,metaclass也就是具有超越变形能力的类
yaml.YAMLObject--只要简单地继承 yaml.YAMLObject，就能让你的 Python Object 具有序列化和逆序列化能力--YAML 的这种动态序列化 / 逆序列化功能正是用 metaclass 实现的
- 实现原理:
    - YAMLObject 把 metaclass 都声明成了 YAMLObjectMetaclass
    - 全局注册器:YAMLObjectMetaclass的__init__函数添加注册逻辑
#### 装饰器和metaclass的区别和联系
    - 都是用于修改类或函数的行为的高级特性
    - 装饰器是一个函数或类,主要作用是在不修改原始代码的情况下,扩展或增强函数功能
    - metaclass的type的子类,用来创建类的类.主要作用是在类被创建的时候动态修改类的行为(属性、方法、继承关系)
#### 设计原理
- 类型模型
    - 所有的python自定义类,都是type类的实例
    - 自定义类,是type类的__call__重载:定义一个类的语句结束时，真正发生的情况，是 Python 调用 type 的__call__运算符--type(classname, superclasses, attributedict)，就是 type 的__call__运算符重载
    - metaclass 是 type 的子类，通过替换 type 的__call__运算符重载机制，“超越变形”正常的类。--一旦你把一个类型 MyClass 的 metaclass 设置成 MyMeta，MyClass 就不再由原生的 type 创建，而是会调用 MyMeta 的__call__运算符重载。


In [1]:
class MyClass:
  data = 1
  
instance = MyClass()
print(MyClass, instance)
print(instance.data)

MyClass = type('MyClass', (), {'data': 1})
instance = MyClass()
print(MyClass, instance)
print(instance.data)

<class '__main__.MyClass'> <__main__.MyClass object at 0x108813c20>
1
<class '__main__.MyClass'> <__main__.MyClass object at 0x10886ca10>
1


In [7]:
class Mymeta(type):
    def __init__(self, name, bases, dic):
        super().__init__(name, bases, dic)
        print('===>Mymeta.__init__')
        print(self.__name__)
        print(dic)
        print(self.yaml_tag)

    def __new__(cls, *args, **kwargs):
        print('===>Mymeta.__new__')
        print(cls.__name__)
        return type.__new__(cls, *args, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('===>Mymeta.__call__')
        obj = cls.__new__(cls)
        cls.__init__(cls, *args, **kwargs)
        return obj
    
class Foo(metaclass=Mymeta):
    yaml_tag = '!Foo'

    def __init__(self, name):
        print('Foo.__init__')
        self.name = name

    def __new__(cls, *args, **kwargs):
        print('Foo.__new__')
        return object.__new__(cls)

class Foo1(Foo):
    yaml_tag = '!Foo'

    def __init__(self, name):
        print('Foo.__init__')
        self.name = name

    def __new__(cls, *args, **kwargs):
        print('Foo.__new__')
        return object.__new__(cls)

===>Mymeta.__new__
Mymeta
===>Mymeta.__init__
Foo
{'__module__': '__main__', '__qualname__': 'Foo', 'yaml_tag': '!Foo', '__init__': <function Foo.__init__ at 0x103f747c0>, '__new__': <function Foo.__new__ at 0x103f74720>}
!Foo
===>Mymeta.__new__
Mymeta
===>Mymeta.__init__
Foo1
{'__module__': '__main__', '__qualname__': 'Foo1', 'yaml_tag': '!Foo', '__init__': <function Foo1.__init__ at 0x103f74900>, '__new__': <function Foo1.__new__ at 0x103f749a0>}
!Foo


In [2]:
# 单例demo
class SingletonMeta(type):
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

# 使用元类创建一个单例类
class SingletonClass(metaclass=SingletonMeta):
    def __init__(self, value):
        self.value = value

# 测试
obj1 = SingletonClass(10)
obj2 = SingletonClass(20)

print(obj1 is obj2)  # True
print(obj1.value)    # 10
print(obj2.value)    # 10

===>Mymeta.__call__
Foo.__new__
Foo.__init__
